Importing necessary libraries / scikitlearn models here 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from keras import layers, models
from keras.models import Sequential
from keras.layers import LSTM, Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn._loss import mean_squared_error

Loading dataset from url link and creating dataframe. Normalizing data here as well by using Sunspots to make prediction and setting a range of 0-1 for the data being represented. Splitting normalized data into 80-20 test-train split.

In [ ]:
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/monthly-sunspots.csv"
df = pd.read_csv(url)

data = df['Sunspots'].values.reshape(-1, 1)

scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

train_size = int(len(scaled_data) * 0.8)
test_data = scaled_data[train_size:]
train_data = scaled_data[:train_size]

Sequence Creation Function (Sliding Window Concept) to analyze current sequence and predict next item for the sequence.

In [ ]:
def create_sequences(dataset, look_back=1):
    x, y = [], []
    for i in range(len(dataset) - look_back - 1):
        a = dataset[i:(i + look_back), 0]
        x.append(a)
        y.append(dataset[i + look_back, 0])
    return np.array(x), np.array(y)

Defining the look back period and reshaping input format to be [samples, time steps, features] for the LSTM RNN Model. Reshaping input for dense to [samples, features] (time steps is flattened)

In [ ]:
# Look Back set to 11 to remain within 60-70 parameter limit, if it were 12 the fully connected model would have 71 parameters
look_back = 11

x_train, y_train = create_sequences(train_data, look_back)
x_test, y_test = create_sequences(test_data, look_back)

# Reshape input for LSTM RNN
x_train_lstm = np.reshape(x_train, (x_train.shape[0],x_train.shape[1], 1))
x_test_lstm = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], 1))

# Reshape for Dense
x_train_dense = x_train
x_test_dense = x_test

Building Both RNN Models

In [ ]:
# --- Model A: Fully Connected RNN Model ---
# 11 Inputs -> 5 Neurons -> 1 Output
# Params: (11*5 + 5) + (5*1 + 1) = 60 + 6 = 66 Parameters
model_fc = Sequential([
    Dense(5, input_dim=look_back, activation=tf.nn.leaky_relu),
    Dense(1)
])
model_fc.compile(optimizer='adam', loss='mse')

# --- Mdel B: LSTM RNN Model ---
# 3 Units
# Params: 4 * ((1 feature + 1 bias) * 3 units + 3 units^2) = 60 LSTM params
# Output Dense: 3 inputs * 1 output + 1 bias = 4 dense params
# Total: 64 Parameters
model_lstm = Sequential([
    LSTM(3, input_shape=(look_back, 1)),
    Dense(1)
])
model_lstm.compile(optimizer='adam', loss='mse')


Verifying Parameters, training Both Models, Creating Predictions / Evals and Inverse Scaling data

In [ ]:
# Verifying Params Counts
print(f"Fully Connected Params: {model_fc.count_params()}")
print(f"LSTM Params: {model_lstm.count_params()}")

# Training Models
print("\nTraining Fully Connected Model...")
model_fc.fit(x_train_dense, y_train, epochs=20, batch_size=32, verbose=1)
print("\nTraining LSTM Model...")
model_lstm.fit(x_train_lstm, y_train, epochs=20, batch_size=32, verbose=1)

# Creating Predictions